# Example 04. Develop a voxel-based model of $P_{gap}$

This example builds a voxel-based model of directional gap probability, $P_{gap}$($\theta$), using multiple scan positions

## Load all the required modules

In [1]:
import os
import numpy as np
import rasterio as rio

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib import animation, rc

from IPython.display import HTML

from pylidar_canopy import voxelization, riegl_io

## Some plotting functions

In [2]:
def initfig(fig, axes, src, layer, vmax_vals, bounds, voxelsize, titles):
    elev = bounds[2] + (layer - 1) * voxelsize
    fig.suptitle(f'Elevation {elev:.1f} m', fontsize=16)
    images = []
    for i,ax in enumerate(axes.flatten()):
        im = ax.imshow(src[i].read(layer), cmap='gist_gray', 
                       animated=True, vmax=vmax_vals[i])        
        xt = ax.get_xticks().tolist()
        yt = ax.get_yticks().tolist()
        t = [src[i].transform * c for c in zip(xt,yt)]
        ax.xaxis.set_major_locator(mticker.FixedLocator(xt))
        ax.set_xticklabels([f'{c[0]:.1f}' for c in t])
        ax.yaxis.set_major_locator(mticker.FixedLocator(yt))
        ax.set_yticklabels([f'{c[1]:.1f}' for c in t])
        ax.set_title(titles[i])
        images.append(im)   
    return images    
    
def updatefig(*args):
    global layer
    elev = bounds[2] + (layer - 1) * voxelsize
    fig.suptitle(f'Elevation {elev:.1f} m', fontsize=16)
    for i,im in enumerate(images):
        im.set_array(src[i].read(layer))
    layer += 1
    return images

## Identify all of the input files

We need both the RDBX and RXP files for the VZ400i and later scanner models. The RDBX have the point cloud corrected with RIEGL MTA processing, and the RXP files have all the pulse information for shots with not returns, allowing us to separate the absence of a return from the absence of a measurement.

If you are using a RIEGL VZ400 scanner or used a pulse rate <= 300khz, then you only need the RXP files.

In [3]:
os.chdir('/gpfs/data1/vclgp/data/tls_point_clouds/riegl_registered/maeda_Amazon_Dimona-100ha-SOUTH.RiSCAN')

# Upright scan files
upright_rxp_fn = 'SCANS/ScanPos001/SINGLESCANS/190426_112319.rxp'
upright_rdbx_fn = 'project.rdb/SCANS/ScanPos001/SINGLESCANS/190426_112319/190426_112319.rdbx'
upright_transform_fn = 'project.rdb/SCANS/ScanPos001.DAT'

# Tilt scan files
tilt_rxp_fn = 'SCANS/ScanPos002/SINGLESCANS/190426_112552.rxp'
tilt_rdbx_fn = 'project.rdb/SCANS/ScanPos002/SINGLESCANS/190426_112552/190426_112552.rdbx'
tilt_transform_fn = 'project.rdb/SCANS/ScanPos002.DAT'

## Process each scan on the same voxel grid

In [4]:
vgrid = voxelization.VoxelGrid() 

In [5]:
vgrid.add_riegl_scan_position_rxp(upright_rxp_fn, upright_transform_fn)

In [10]:
bounds = [-50,-50,85,50,50,130]
voxelsize = 1.0

vgrid.voxelize_scan(bounds, voxelsize)

In [11]:
prefix = '/gpfs/data1/vclgp/armstonj/tls_temp/190426_112319'
vgrid.write_grids(prefix)

## Visualize the single scan voxel outputs

In [12]:
fig,axes = plt.subplots(figsize=[15,4], ncols=3, nrows=1)

src = [rio.open(vgrid.filenames[k]) for k in sorted(vgrid.filenames)]
vmax_vals = [1,5,1]
titles = [r'$P_{gap}$','Class','Weight']

layer = 1
images = initfig(fig, axes, src, layer, vmax_vals, bounds, voxelsize, titles)
anim = animation.FuncAnimation(fig, updatefig, interval=50, blit=True, frames=42)
plt.close()

HTML(anim.to_jshtml())